In [1]:
require 'hdf5'

f = hdf5.open("data.hdf5", "r")

X_train = f:read("train_X_sequence"):all()
Y_train = f:read("train_Y"):all()
X_valid = f:read("valid_reduced_X"):all()
Y_valid = f:read("valid_reduced_Y"):all()

nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]

f:close()

In [7]:
--Converts LongTensor into String for table hashing
function createHash(longTensor)
    s = ""
    for i=1,longTensor:size(1) do
        s = s .. tostring(longTensor[i]) .. " "
    end
    return s
end

--Returns table of space probabilities
function count(char_stream, space_stream, n_gram)
    local space_dic = {}
    local count_dic = {}
    for i=1,char_stream:size(1)-n_gram+1 do
        local pre = createHash(char_stream:sub(i,i+n_gram-1))
        local nex = space_stream:sub(i+n_gram-1,i+n_gram-1)[1]

        if count_dic[pre] then
            count_dic[pre] = count_dic[pre] + 1
        else
            count_dic[pre] = 1
        end

        if nex == 2 then
            if space_dic[pre] then
                space_dic[pre] = space_dic[pre] + 1
            else
                space_dic[pre] = 1
            end
        end
    end
    for k,_ in pairs(space_dic) do
        space_dic[k] = space_dic[k]/count_dic[k]
    end
    return space_dic
end

In [22]:
dic = count(X_train, Y_train, 5)

In [24]:
for i=1, 15 do
    local pre = createHash(X_valid:sub(i,i+5-1))
    if dic[pre] then
        print(pre, dic[pre])
    else
        print(pre, 0)
    end
end

2 14 15 18 20 	0	
14 15 18 20 12 	0	
15 18 20 12 4 	0	
18 20 12 4 19 	0.64864864864865	
20 12 4 19 18 	0.91666666666667	
12 4 19 18 28 	0	
4 19 18 28 12 	0	
19 18 28 12 1 	0	
18 28 12 1 24 	0.9047619047619	
28 12 1 24 28 	0	
12 1 24 28 22 	0	
1 24 28 22 1 	0	
24 28 22 1 15 	0	
28 22 1 15 21 	0.52857142857143	
22 1 15 21 28 	0	


In [25]:
X_valid:sub(1,19)

  2
 14
 15
 18
 20
 12
  4
 19
 18
 28
 12
  1
 24
 28
 22
  1
 15
 21
 28
[torch.LongTensor of size 19]

